BTM_bitermplus
========

Model fitting
-------------



In [26]:
import pandas as pd

In [27]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# read the data that we prepared for analysis
df = pd.read_csv('all_respones_for_all_interviewees_respones_LIWC_words_V2.csv')
print(df.shape)
print(df.columns)



(17953, 8)
Index(['ids', 'diagnosis', 'section', 'pilot_number', 'inter_time', 'response',
       'Nresponse', 'LIWC_response'],
      dtype='object')


In [28]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default POS tag
    

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lilifang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lilifang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
import re
from nltk.tokenize import word_tokenize
# from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import string
import numpy as np
import pandas as pd


def clean_text2(sentence): 
    pattern = re.compile(r'[^a-z]+')
    sentence = sentence.lower()
    sentence = pattern.sub(' ', sentence).strip()
    # Tokenize
    word_list = word_tokenize(sentence)
    # stop words
    stopwords_list = stopwords.words('english') 
    new_stopwords = ["um", "oh", "okay", "mhm","gon", "na","ah","uh","yes",'yep',"yeah","no","hm","wow",'.','?','-','--',':','mm','oop']
    #new_stopwords = ["um", "oh", "okay", "mhm","ah","uh","yes","get","yeah","no","hm","wow",'.','?','-','--',':','mm']
    stopwords_list.extend(new_stopwords)   
    # puctuation
    punct = set(string.punctuation)
    # remove stop words
    word_list = [word for word in word_list if word not in stopwords_list]
    # remove very small words, length < 3
    # they don't contribute any useful information
    word_list = [word for word in word_list if len(word) >= 1]
    # remove punctuation
    word_list = [word for word in word_list if word not in punct] 
    
    # Define the corpus of documents as a list of strings
    lemmatizer = WordNetLemmatizer()
    
    pos_tags = pos_tag(word_list)

    lemmas = []
    for word, tag in pos_tags:
        wordnet_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(word, pos=wordnet_tag)
        lemmas.append(lemma)

    lemmatized_sentence = " ".join(lemmas)
    #print(lemmatized_sentence) 
    return lemmatized_sentence


In [30]:
df['response_lemar'] = df['response'].apply(lambda x: clean_text2(str(x)))
# df['response_stem'] = df['response_stem'].apply(lambda x: ' '.join(x))

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Define the parameters
min_df = 0.0025
max_df = 0.5
# Create a CountVectorizer object
vectorizer = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df)
# Fit the vectorizer to the text data
X = vectorizer.fit_transform(df['response_lemar'])
# Get the feature names (i.e., the words)
feature_names = vectorizer.get_feature_names_out()

# Convert the sparse matrix to a dense matrix
dense_matrix = X.toarray()

# Create a new dataframe with the filtered words
filtered_words_df = pd.DataFrame(data=dense_matrix, columns=feature_names)

# Add the new column to the original dataframe
df['filtered_text_0.0025'] = filtered_words_df.apply(lambda x: ' '.join([word for word, count in zip(feature_names, x) if count > 0]), axis=1)


In [32]:
df['words_count'] = df['response'].str.split().str.len()
df=df[df['words_count']>2]
print(df.shape)

(13957, 11)


In [33]:
# !pip install bitermplus
import bitermplus as btm
import numpy as np
import pandas as pd

In [34]:
old_texts = df['filtered_text_0.0025'].str.strip().tolist()

In [35]:
print(old_texts[0:5])

['like say', 'say', 'mum', '', 'cause forget like picture thought want']


Index(['ids', 'diagnosis', 'section', 'pilot_number', 'inter_time', 'response',
       'Nresponse', 'LIWC_response', 'words_count', 'response_lemar',
       'filtered_text_0.0025'],
      dtype='object')


In [36]:
# Tokenize the text into words
tokens = []

for item in old_texts:
    try:
        new_i = item.split(' ')
        # print(new_i)
        for i in new_i:
            tokens.append(i)
    except:
        pass

# Create a set to store unique words
unique_words = set()

# Iterate through the tokens and add them to the set
for token in tokens:
    unique_words.add(token)

# Count the unique words
num_unique_words = len(unique_words)

print("Number of distinct words:", num_unique_words)

# before lemmatize Number of distinct words: 8432  
# after lemmatize, number of distinct words: 4071
# filter lemmatize, number of distanct words: 0.01, 85
# filtered_text_0.005: 174
# filtered_text_0.0025: 306

Number of distinct words: 306


In [37]:
print(tokens[0:5])

['like', 'say', 'say', 'mum', '']


In [38]:
display(df[['response','response_lemar','filtered_text_0.0025']].head(10))

,response,response_lemar,filtered_text_0.0025
3,Just says like,say like,like say
4,Okay. There's something you said. Something,something say something,say
7,Just my mum so --,mum,mum
8,You can't be out here,,
9,With the picture cause I had to rush cause I ...,picture cause rush cause forget like like thou...,cause forget like picture thought want
11,I do dance on Saturday I do singing at home e...,dance saturday sing home every day do draw sin...,day draw home read
12,So I'm reading this book by an author called R...,read book author call rachel renee russell cal...,book like read
13,It's about this girl. It's in the form of a di...,girl form diary speaking trouble school good t...,able book finish good happen school thing trouble
15,What? The book or about me?,book,book
16,So acting childish often struggling with a lo...,act childish often struggle lot thing think tw...,act lot struggle thing think


In [42]:
print(df.shape)
df=df.dropna(subset=['filtered_text_0.0025'])
print(df.shape)

(13957, 12)
(13957, 12)


In [43]:
df['index'] = pd.DataFrame(range(df.shape[0]))

In [44]:
texts = df['filtered_text_0.0025'].str.strip().tolist()

In [21]:
# import nltk
# nltk.download('punkt')
# stopwords = nltk.corpus.stopwords.words('english')
# new_stopwords = ["um", "oh", "okay", "mhm","ah","uh","yes","get",'yep',"yeah","no","hm","wow",'.','?','-','--',':','mm','Mm','Oop']
# stopwords.extend(new_stopwords)

In [45]:
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

In [46]:
display(len(texts),len(docs_vec),len(biterms))

13957

13957

10897

In [47]:
# finding best topic_num
   
metrics=[]
for topic_num in range(2,15):
    print(topic_num)
    model = btm.BTM(X, vocabulary, seed=124, T=topic_num, M=20, alpha=50/10, beta=0.01)
    model.fit(biterms, iterations=20)
    # Get a phi matrix
    #phi = tmp.get_phi(model)
    #entropy = tmp.entropy(phi)
    perplexity = model.perplexity_
    coherence = model.coherence_
    metrics.append([topic_num,perplexity,coherence])
metrics=pd.DataFrame(metrics)
metrics.columns=['Topic number', 'perplexity','coherence']
metrics['coherence mean']=[np.mean(x) for x in metrics['coherence'].tolist()]
  

2


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 84.20it/s]


3


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 80.43it/s]


4


...

100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 45.24it/s]


13


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 43.33it/s]


14


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 41.23it/s]


##  Choosing the Best Coherence Score
There is no one way to determine whether the coherence score is good or bad. The score and its value depend on the data that it’s calculated from. For instance, in one case, the score of 0.5 might be good enough but in another case not acceptable. The only rule is that we want to maximize this score.

Usually, the coherence score will increase with the increase in the number of topics. This increase will become smaller as the number of topics gets higher. The trade-off between the number of topics and coherence score can be achieved using the so-called elbow technique. The method implies plotting coherence score as a function of the number of topics. We use the elbow of the curve to select the number of topics.

The idea behind this method is that we want to choose a point after which the diminishing increase of coherence score is no longer worth the additional increase of the number of topics. The example of elbow cutoff at n\_topics = 3 is shown below:

In [48]:
display(pd.DataFrame(metrics)) 

,Topic number,perplexity,coherence,coherence mean
0,2,inf,"[-439.5016097180295, -438.78047975119284]",-439.141045
1,3,3.381857e+260,"[-441.3578568481386, -473.63010635683526, -440...",-451.703858
2,4,3.869143e+273,"[-453.7403830509875, -460.60543369677407, -443...",-450.630384
3,5,1.379481e+11,"[-456.6900096787568, -472.4628861203783, -487....",-462.502802
4,6,1.058289e+63,"[-467.4566694227514, -434.24939313945, -479.22...",-457.580639
5,7,5.023411e+100,"[-473.12810145276944, -442.8799558432416, -490...",-467.542055
6,8,3.204137e+129,"[-468.46545716516664, -462.57909828084223, -44...",-470.191901
7,9,6.125661e+149,"[-444.401258455495, -469.1048875485441, -495.2...",-476.566894
8,10,1.000000e+00,"[-467.06880940193355, -482.0960872775805, -459...",-477.596970
9,11,2.914873e+01,"[-470.62499241095645, -462.9192613408833, -472...",-471.663902


In [49]:
import tmplot as tmp
optimal_topic_number=10
model = btm.BTM(X, vocabulary, seed=1234, T=optimal_topic_number, M=20)
model.fit(biterms, iterations=500)
# Get a phi matrix
phi = tmp.get_phi(model)
# entropy = tmp.entropy(phi)

100%|█████████████████████████████████████████| 500/500 [00:09<00:00, 53.52it/s]
/opt/anaconda3/lib/python3.8/site-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(


In [50]:
p_zd = model.transform(docs_vec)
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 10)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)
print(perplexity)

100%|█████████████████████████████████| 13957/13957 [00:00<00:00, 204106.19it/s]

117.15953029362926


In [51]:
vocab_dict_pd=pd.DataFrame(vocab_dict.items(), columns=['Word', 'Index'])
display(vocab_dict_pd)

,Word,Index
0,like,146
1,say,229
2,mum,176
3,cause,38
4,forget,95
...,...,...
300,night,183
301,sense,234
302,couple,51
303,dont,66


In [52]:
import tmplot as tmp

In [53]:
print(model.labels_)

[9 9 9 ... 5 5 4]


In [54]:
#Print the top 15 words for each topic
print(len(model.labels_))
labels = pd.DataFrame(model.labels_)
new_id = pd.DataFrame(range(len(model.labels_)))
df_with_label=pd.DataFrame()
df_with_label['index']=new_id
df_with_label['topic_label']=labels

13957


In [55]:
print(df_with_label.shape)

(13957, 2)


In [56]:
df['index']=range(0,len(df))
print(df.shape)

(13957, 12)


In [57]:
new_df_with_label = pd.merge(df,df_with_label,on='index',how='left')
print(new_df_with_label.shape)

(13957, 13)


In [59]:
nf =open('topic_words.csv','wt')
nf.write('topic,reprensentative_words'+'\n')
for i in range(10):
    terms_probs = tmp.calc_terms_probs_ratio(phi, topic=i, lambda_=1)
    print( terms_probs['Terms'][0:20])
    words = ' '.join(row for row in terms_probs['Terms'][0:20])
    print(i,words)
    nf.write(str(i)+','+str(words)+'\n')

0        like
1       thing
2      really
3         lot
4      people
5        time
6       quite
7       think
8        make
9        calm
10      place
11       feel
12    usually
13        try
14       help
15      focus
16        bit
17      stuff
18      noise
19       sort
Name: Terms, dtype: object
0 like thing really lot people time quite think make calm place feel usually try help focus bit stuff noise sort
0          like
1     attention
2           pay
3        really
4           try
5          talk
6      distract
7         focus
8         think
9        lesson
10          lot
11       friend
12         work
13       people
14          say
15         know
16         kind
17      teacher
18        thing
19        quite
Name: Terms, dtype: object
1 like attention pay really try talk distract focus think lesson lot friend work people say know kind teacher thing quite
0         like
1         make
2        thing
3         feel
4        think
5         time
6         know
7     

In [60]:
topic_words_df = pd.read_csv('topic_words.csv')
print(topic_words_df.shape)
print(topic_words_df.columns)
topic_words_df. rename(columns = {'topic':'topic_label'}, inplace = True)
print(topic_words_df.columns)

(10, 2)
Index(['topic', 'reprensentative_words'], dtype='object')
Index(['topic_label', 'reprensentative_words'], dtype='object')


In [164]:
df_with_label_topicwords = pd.merge(new_df_with_label,topic_words_df,on='topic_label',how='left')
display(df_with_label_topicwords.head())
display(df_with_label_topicwords.shape)

,ids,diagnosis,section,pilot_number,inter_time,response,Nresponse,LIWC_response,response_lemar,filtered_text_0.0025,words_count,index,topic_label,reprensentative_words
0,3,ASD.ADHD,Activity Three,974_16_06_2022,[00:01:08],Just says like,True,says,say like,like say,3,0,9,like say really know think feel start people t...
1,4,ASD.ADHD,Activity Three,974_16_06_2022,[00:01:18],Okay. There's something you said. Something,True,something you,something say something,say,6,1,9,like say really know think feel start people t...
2,7,ASD.ADHD,Icebreaker,974_16_06_2022,[00:03:16],Just my mum so --,True,mum,mum,mum,5,2,9,like say really know think feel start people t...
3,8,ASD.ADHD,Icebreaker,974_16_06_2022,[00:03:53],You can't be out here,True,can't,,,5,3,0,like thing really lot people time quite think ...
4,9,ASD.ADHD,Icebreaker,974_16_06_2022,[00:04:16],With the picture cause I had to rush cause I ...,True,picture didn't thought but couldn't wanted,picture cause rush cause forget like like thou...,cause forget like picture thought want,38,4,9,like say really know think feel start people t...


(13957, 14)

In [165]:
df_with_label_topicwords['new_words_count'] = df_with_label_topicwords['filtered_text_0.0025'].str.split().str.len()
df_with_label_topicwords=df_with_label_topicwords[df_with_label_topicwords['new_words_count']>0]

In [166]:
print(df_with_label_topicwords.shape)
df_with_label_topicwords.to_csv('filtered_text_final_0.0025.csv',index=False)

(13208, 15)


In [63]:
new_count_df= df_with_label_topicwords.groupby('topic_label',as_index=False).count()
display(new_count_df[['topic_label','ids']])

,topic_label,ids
0,0,950
1,1,260
2,2,460
3,3,666
4,4,2908
5,5,1790
6,6,655
7,7,1201
8,8,487
9,9,4580


In [64]:
from typing import List, Union, Tuple, Dict, Sequence, Any
def get_biterms_new(
        docs: List[np.ndarray],
        win: int = 15) -> List[List[int]]:
    biterms = []
    doc_index=0
    for doc in docs:
        doc_biterms = []
        doc_len = len(doc)
        for i in range(doc_len-1):
            for j in range(i+1, min(i + win, doc_len)):
                wi = min(doc[i], doc[j])
                wj = max(doc[i], doc[j])
                doc_biterms.append([wi, wj])
        biterms.append([doc_index,doc_biterms])
        doc_index=doc_index+1
    return biterms


In [65]:
biterms_new = get_biterms_new(docs_vec)

In [173]:
biterms_new=pd.DataFrame(biterms_new)
biterms_new.columns=['index','biterms in doc']
display(biterms_new)

,index,biterms in doc
0,0,"[[146, 229]]"
1,1,[]
2,2,[]
3,3,[]
4,4,"[[38, 95], [38, 146], [38, 200], [38, 270], [3..."
...,...,...
13952,13952,[]
13953,13953,[]
13954,13954,[]
13955,13955,"[[85, 218]]"


In [174]:
label_number = 1
doc_id_in_topic=df_with_label_topicwords['index'][df_with_label_topicwords['topic_label']==label_number].values.tolist()
# doc_id_in_topic

In [175]:
BitermDataDoc=biterms_new['biterms in doc'][biterms_new['index'].isin(doc_id_in_topic)].values.tolist()
# BitermDataDoc[0]

In [176]:
BitermsSetDoc=[]
for row in BitermDataDoc:
    BitermsSetDoc=BitermsSetDoc+row
BitermsSetDoc=pd.DataFrame(BitermsSetDoc)   
BitermsSetDoc.columns=['O','D']
BitermsSetDoc=BitermsSetDoc[BitermsSetDoc['O']!=BitermsSetDoc['D']]
display(BitermsSetDoc.head(3))

,O,D
0,17,195
1,17,263
2,17,287


In [177]:
BitermsSetDoc_count= BitermsSetDoc[['O','D']].value_counts().reset_index()

In [178]:
display(BitermsSetDoc_count.head(3))

,O,D,0
0,17,195,103
1,146,195,41
2,17,146,41


In [179]:
vocab_dict_df=pd.DataFrame(vocab_dict.items(), columns=['O word', 'O'])
vocab_dict_df['D']=vocab_dict_df['O']
vocab_dict_df['D word']=vocab_dict_df['O word']
BitermsSetDoc_count=pd.merge(BitermsSetDoc_count, vocab_dict_df[['O word', 'O']], how='left', on='O')
BitermsSetDoc_count=pd.merge(BitermsSetDoc_count, vocab_dict_df[['D word', 'D']], how='left', on='D')
# display(BitermsSetDoc_count)

In [180]:
BitermsSetDoc_count['Edge weight']=BitermsSetDoc_count[0]
top_BitermsSetDoc_count=BitermsSetDoc_count.sort_values(by='Edge weight',ascending=False).head(1000)
# top_BitermsSetDoc_count

In [181]:
import networkx as nx
def plotNetwork(df):
    G = nx.DiGraph()
    for index, row in df.iterrows():
        G.add_edge(row['O word'], row['D word'], weight=row['Edge weight'])  
    return G

In [182]:
color_lis = ['lightblue','pink','lightskyblue','darkseagreen','gold',
             'bisque','tomato','yellowgreen','royalblue','plum']

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
path1= './BTM-biterms-filtered-data-summary/filter_text_final_0.0025/'
# Create a new figure and subplots
Net_data =top_BitermsSetDoc_count[["O word","D word","Edge weight"]]
Net_data_new=Net_data
print(Net_data_new.shape)
G0 = plotNetwork(Net_data_new)
#nx.write_gml(G0, 'topic2.gml')
weights = dict(G0.degree(weight='weight'))
# Sort the dictionary by weights in descending order
sorted_weights = {k: v for k, v in sorted(weights.items(), key=lambda item: item[1], reverse=True)}
# Keep only the top 20 words (nodes) with largest weights
top_words = list(sorted_weights.keys())[0:50]
# Create a new network with only the top 20 words and their corresponding edges
#plt.savefig("Graph_topic_"+str(i)+".png", format="PNG")
new_G = G0.subgraph(top_words)
nx.write_gml(new_G, path1+str(label_number)+'_for_Re-star.gml')

top_BitermsSetDoc_count.to_csv(path1+'filter_text_final_0.0025_biterms_summary'+str(label_number)+'.csv',index=False)

(1000, 3)
